## PyTorch 선형 회귀 구현

In [1]:
import pandas as pd
import numpy as np 
import torch 

import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
torch.manual_seed(7777)

In [6]:
from sklearn import datasets
X, y = datasets.fetch_openml('boston', return_X_y=True)
X

c:\Users\nabi4\anaconda3\envs\ds_study\lib\site-packages\sklearn\datasets\_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
c:\Users\nabi4\anaconda3\envs\ds_study\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48


In [8]:
df = X.copy()
df['const'] = np.ones(df.shape[0])
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273.0,21.0,396.90,7.88,1.0


In [9]:
df.shape

(506, 14)

먼저, x의 역행렬이 존재한다면 선형회귀에서의 가중치 벡터를 구할 수 있는 계산을 한 번 시도해보자.

https://datascienceschool.net/03%20machine%20learning/04.02%20%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D%EC%9D%98%20%EA%B8%B0%EC%B4%88.html?highlight=intercept

수식 설명 참고

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   CRIM     506 non-null    float64 
 1   ZN       506 non-null    float64 
 2   INDUS    506 non-null    float64 
 3   CHAS     506 non-null    category
 4   NOX      506 non-null    float64 
 5   RM       506 non-null    float64 
 6   AGE      506 non-null    float64 
 7   DIS      506 non-null    float64 
 8   RAD      506 non-null    category
 9   TAX      506 non-null    float64 
 10  PTRATIO  506 non-null    float64 
 11  B        506 non-null    float64 
 12  LSTAT    506 non-null    float64 
 13  const    506 non-null    float64 
dtypes: category(2), float64(12)
memory usage: 49.0 KB


In [18]:
# object 데이터타입 때문에 tensor로 바뀌지 않는 오류를 해결하기 위함
df['CHAS'] = df['CHAS'].astype('float32')
df['RAD'] = df['RAD'].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float32
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float32
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  const    506 non-null    float64
dtypes: float32(2), float64(12)
memory usage: 51.5 KB


In [16]:
y.dtype

dtype('float64')

In [19]:
x = torch.tensor(df.values)
y = torch.tensor(y.values).view(-1, 1)

XT = torch.transpose(x, 0, 1) 
# 역행렬이 존재시, 가중치 벡터 구하는 수식
w = torch.matmul(torch.matmul(torch.inverse(torch.matmul(XT, x)), XT), y)
y_pred = torch.matmul(x, w)

print("예측한 집값 :", y_pred[19], "실제 집값 :", y[19])

예측한 집값 : tensor([18.4061], dtype=torch.float64) 실제 집값 : tensor([18.2000], dtype=torch.float64)


#### Gradient descent 방식

In [20]:
# 가중치 생성
# const 열을 추가했기 때문에 이론 상으로 b는 필요없지만
# 그냥 일반적인 경우를 설명하기 위해 맞춰준 것
w = torch.rand((14, 1), dtype=torch.float64, requires_grad=True)
b = torch.rand((1, 1),  dtype=torch.float64, requires_grad=True)

In [21]:
z = x.mm(w) + b
loss = torch.mean((z - y)**2) 

In [22]:
loss.backward()
w.grad, b.grad

(tensor([[4.6543e+03],
         [1.1592e+04],
         [1.2824e+04],
         [7.4684e+01],
         [6.1311e+02],
         [6.7807e+03],
         [7.6970e+04],
         [3.9565e+03],
         [1.1487e+04],
         [4.6771e+05],
         [2.0181e+04],
         [3.9043e+05],
         [1.4341e+04],
         [1.0850e+03]], dtype=torch.float64),
 tensor([[1084.9567]], dtype=torch.float64))

In [23]:
print(loss)

tensor(303557.1678, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [25]:
# loss 숫자만 추출하는 방법
print(loss.detach().numpy())  # 방법 1

with torch.no_grad(): # 방법 2
    print(loss.numpy())

# 이렇게 로그를 추출할 때는 숫자만 추출하도록 설정해줌

303557.16782532405
303557.16782532405


In [26]:
# 방법 3
loss.item()

303557.16782532405

#### assign 대신에 data에 접근해서 값을 수정 

tensor.data = 다른데이터

In [27]:
w

tensor([[0.9625],
        [0.8798],
        [0.6916],
        [0.0976],
        [0.5441],
        [0.6069],
        [0.7656],
        [0.2674],
        [0.7172],
        [0.4656],
        [0.7843],
        [0.7482],
        [0.5919],
        [0.0070]], dtype=torch.float64, requires_grad=True)

In [28]:
w.data # 수정이 가능함!

tensor([[0.9625],
        [0.8798],
        [0.6916],
        [0.0976],
        [0.5441],
        [0.6069],
        [0.7656],
        [0.2674],
        [0.7172],
        [0.4656],
        [0.7843],
        [0.7482],
        [0.5919],
        [0.0070]], dtype=torch.float64)

In [29]:
lr = 3e-7
for epoch in range(100):
    z = x.mm(w) + b
    loss = torch.mean((y-z)**2)

    loss.backward() # 미분값 계산

    w.data -= w.grad * lr # 업데이트
    b.data -= b.grad * lr

    print('{} - loss : {}'.format(epoch, loss.item()))

    # 초기화
    # 누적값을 초기화해주어야함
    w.grad.zero_()
    b.grad.zero_()

0 - loss : 303557.16782532405
1 - loss : 119223.98659357797
2 - loss : 79124.14612788575
3 - loss : 52663.835776943364
4 - loss : 35201.07677258205
5 - loss : 23673.781087933094
6 - loss : 16062.035864032894
7 - loss : 11033.399307362253
8 - loss : 7708.920110912624
9 - loss : 5508.792152027679
10 - loss : 4050.540603569035
11 - loss : 3081.8603939022364
12 - loss : 2436.3130901562417
13 - loss : 2004.102341341875
14 - loss : 1712.7958941445906
15 - loss : 1514.6088658572062
16 - loss : 1378.015150935417
17 - loss : 1282.2138811644036
18 - loss : 1213.4792221507962
19 - loss : 1162.7525127944627
20 - loss : 1124.0539164537597
21 - loss : 1093.4346631281337
22 - loss : 1068.285892621277
23 - loss : 1046.8827296477355
24 - loss : 1028.0835297906758
25 - loss : 1011.1314840058286
26 - loss : 995.5237440302692
27 - loss : 980.9250880422532
28 - loss : 967.1109673806691
29 - loss : 953.929934560665
30 - loss : 941.278856238908
31 - loss : 929.0865598461065
32 - loss : 917.3030435611203
33 -

In [30]:
lr = 3e-7
for epoch in range(100):
    z = x.mm(w) + b
    loss = torch.mean((y-z)**2)

    # 미분값 계산
    grads = torch.autograd.grad(loss, [w, b])

    w.data -= grads[0] * lr # 업데이트
    b.data -= grads[1] * lr

    print('{} - loss : {}'.format(epoch, loss.item()))

    # 초기화
    # 누적값을 초기화해주어야함
    w.grad.zero_()
    b.grad.zero_()

0 - loss : 565.768095947369
1 - loss : 563.9983008315795
2 - loss : 562.2739253453589
3 - loss : 560.59370131941
4 - loss : 558.9563960453835
5 - loss : 557.3608112842761
6 - loss : 555.8057823025581
7 - loss : 554.2901769352537
8 - loss : 552.8128946752205
9 - loss : 551.3728657878954
10 - loss : 549.9690504507961
11 - loss : 548.600437917083
12 - loss : 547.266045702513
13 - loss : 545.9649187951267
14 - loss : 544.6961288870357
15 - loss : 543.4587736276931
16 - loss : 542.2519758980415
17 - loss : 541.0748831049638
18 - loss : 539.9266664954564
19 - loss : 538.8065204899864
20 - loss : 537.7136620344866
21 - loss : 536.647329970473
22 - loss : 535.6067844227732
23 - loss : 534.5913062043794
24 - loss : 533.6001962379404
25 - loss : 532.6327749934328
26 - loss : 531.6883819415552
27 - loss : 530.7663750224079
28 - loss : 529.8661301290299
29 - loss : 528.9870406053782
30 - loss : 528.1285167583453
31 - loss : 527.2899853834207
32 - loss : 526.4708893036186
33 - loss : 525.6706869212

#### optimizer 사용하기

In [31]:
opt = torch.optim.SGD([w, b], lr=lr)

for epoch in range(100):    
    z = (x.mm(w) + b)
    loss = torch.mean((z - y)**2)  
    
    loss.backward()
    opt.step()
    print("{:3} - loss : {}".format(epoch, loss.item()), end="\r")
    
    opt.zero_grad()

In [33]:
y_pred = x.mm(w) + b
print("예측한 집값 :", y_pred[19].item(), "실제 집값 :", y[19].item())

예측한 집값 : 22.069598463384597 실제 집값 : 18.2
